In [11]:
from src import utils
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from src import model as models

ModuleNotFoundError: No module named 'restrictedSoftmax'

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
# 텐서플로가 첫 번째 GPU에 1GB 메모리만 할당하도록 제한
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024 * 10)])
    except RuntimeError as e:
        print(e)

In [4]:
datafile_path = os.path.join(os.path.abspath(os.getcwd()), 
                        ('data/server/')) 

In [5]:
print(datafile_path)

/data_disk/home/joongho/FL/data/server/


In [6]:
train_DS, val_DS = utils.data_loader(datafile_path)

Found 50000 files belonging to 10 classes.
Using 40000 files for training.
Found 50000 files belonging to 10 classes.
Using 10000 files for validation.


In [ ]:
general_mode = utils.mode